In [2]:
from sqlalchemy import Column, Integer, String, Table, create_engine
from sqlalchemy import orm, MetaData, Column, ForeignKey
from sqlalchemy.orm import relation, mapper, sessionmaker
from sqlalchemy.orm.collections import column_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///:memory:', echo=True)
Base = declarative_base(bind=engine)

class Note(Base):
    __tablename__ = 'notes'

    id_item = Column(Integer, ForeignKey('items.id'), primary_key=True)
    name = Column(String(20), primary_key=True)
    value = Column(String(100))

    def __init__(self, name, value):
        self.name = name
        self.value = value        

class Item(Base):
    __tablename__ = 'items'
    id = Column(Integer, primary_key=True)
    name = Column(String(20))
    description = Column(String(100))
    _notesdict = relation(Note, 
                          collection_class=column_mapped_collection(Note.name))
    notes = association_proxy('_notesdict', 'value', creator=Note)

    def __init__(self, name, description=''):
        self.name = name
        self.description = description

Base.metadata.create_all()

2016-07-16 22:46:36,802 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-07-16 22:46:36,803 INFO sqlalchemy.engine.base.Engine ()
2016-07-16 22:46:36,804 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-07-16 22:46:36,805 INFO sqlalchemy.engine.base.Engine ()
2016-07-16 22:46:36,806 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("items")
2016-07-16 22:46:36,807 INFO sqlalchemy.engine.base.Engine ()
2016-07-16 22:46:36,808 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("notes")
2016-07-16 22:46:36,809 INFO sqlalchemy.engine.base.Engine ()
2016-07-16 22:46:36,810 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE items (
	id INTEGER NOT NULL, 
	name VARCHAR(20), 
	description VARCHAR(100), 
	PRIMARY KEY (id)
)


2016-07-16 22:46:36,811 INFO sqlalchemy.engine.base.Engine ()
2016-07-16 22:46:36,812 INFO sqlalchemy.engine.base.Engine COMMIT
2016-07-16 22:46:36,813 INFO sqlalchemy.

In [3]:
Session = sessionmaker(bind=engine)
s = Session()

i = Item('ball', 'A round full ball')
i.notes['color'] = 'orange'
i.notes['size'] = 'big'
i.notes['data'] = 'none'

s.add(i)
s.commit()
print i.notes

2016-07-16 22:48:09,042 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-07-16 22:48:09,044 INFO sqlalchemy.engine.base.Engine INSERT INTO items (name, description) VALUES (?, ?)
2016-07-16 22:48:09,045 INFO sqlalchemy.engine.base.Engine ('ball', 'A round full ball')
2016-07-16 22:48:09,047 INFO sqlalchemy.engine.base.Engine INSERT INTO notes (id_item, name, value) VALUES (?, ?, ?)
2016-07-16 22:48:09,048 INFO sqlalchemy.engine.base.Engine ((1, 'color', 'orange'), (1, 'data', 'none'), (1, 'size', 'big'))
2016-07-16 22:48:09,049 INFO sqlalchemy.engine.base.Engine COMMIT
2016-07-16 22:48:09,051 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-07-16 22:48:09,052 INFO sqlalchemy.engine.base.Engine SELECT items.id AS items_id, items.name AS items_name, items.description AS items_description 
FROM items 
WHERE items.id = ?
2016-07-16 22:48:09,053 INFO sqlalchemy.engine.base.Engine (1,)
2016-07-16 22:48:09,055 INFO sqlalchemy.engine.base.Engine SELECT notes.id_item AS notes_id

In [4]:
i = Item('ball', 'A round full ball')
i.notes['color'] = 'orange'
i.notes['size'] = 'small'
i.notes['data'] = 'none'

s.add(i)
s.commit()
print i.notes

2016-07-16 22:48:58,773 INFO sqlalchemy.engine.base.Engine INSERT INTO items (name, description) VALUES (?, ?)
2016-07-16 22:48:58,774 INFO sqlalchemy.engine.base.Engine ('ball', 'A round full ball')
2016-07-16 22:48:58,775 INFO sqlalchemy.engine.base.Engine INSERT INTO notes (id_item, name, value) VALUES (?, ?, ?)
2016-07-16 22:48:58,776 INFO sqlalchemy.engine.base.Engine ((2, 'color', 'orange'), (2, 'data', 'none'), (2, 'size', 'small'))
2016-07-16 22:48:58,777 INFO sqlalchemy.engine.base.Engine COMMIT
2016-07-16 22:48:58,778 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-07-16 22:48:58,779 INFO sqlalchemy.engine.base.Engine SELECT items.id AS items_id, items.name AS items_name, items.description AS items_description 
FROM items 
WHERE items.id = ?
2016-07-16 22:48:58,780 INFO sqlalchemy.engine.base.Engine (2,)
2016-07-16 22:48:58,782 INFO sqlalchemy.engine.base.Engine SELECT notes.id_item AS notes_id_item, notes.name AS notes_name, notes.value AS notes_value 
FROM notes 
W

In [6]:
for i in s.query(Note).all():
    print i.name, i.value
    

2016-07-16 22:51:25,652 INFO sqlalchemy.engine.base.Engine SELECT notes.id_item AS notes_id_item, notes.name AS notes_name, notes.value AS notes_value 
FROM notes
2016-07-16 22:51:25,653 INFO sqlalchemy.engine.base.Engine ()
color orange
data none
size big
color orange
data none
size small
